# PhiFlow Assignment

## 1. Problem Statement
Figure 1 shows a 2D incompressible fluid domain with two obstacles forming a boundary within the domain. We want to determine the right fluid inflow on the bottom boundary in order for the fluid to exit at speed 1.0 from the top boundary.
The fluid is incompressible and has negligible viscosity.


<center>
<img src="https://i.imgur.com/W2V2gQl.png"/>
</center>

In [ ]:
from phi.torch.flow import *  # I used the PyTorch backend, but you can use the Jax or TensorFlow backend if you prefer

## 2. Constructing geometry
The flow of the fluid is guided by two obstacles, but their geometry is not able to be represented with the simple geometries provided by PhiFlow.
We can, however, build this geometry by defining an additional Geometry primitive which we will combine with the existing `Box` Geometry.

Define a class `Triangle` that extends the `Geometry` class. The `Triangle` class should have the following methods:
* An `__init__` constructor that takes three points as its input. You may assume that they are given in counter-clockwise order, and don't overlap. Your constructor should store these points as a single `Tensor` with shape `instance('vertex')` and `channel(vector='x,y')`.
* Properties `shape`, `volume`, `shape_type`, and `center`. These are functions with no arguments that are decorated with the `@property` decorator.
* Methods `__variable_attrs__`, `lies_inside` (use barycentric coordinates), `bounding_radius`, and `bounding_half_extent`.
* The most difficult method to implement will be `approximate_signed_distance`. Make sure your code is entirely vectorized. There should be no loops in your solution. Make sure that the result is the _signed_ distance. You can find the formula for the distance between a line segment and a point online.

Hint: Look at the implementation of `Box` and `Sphere` in `phi/geom/geometry.py`.
`__variable_attrs__` can return `('_points',)` as a tuple.
For implementing `approximate_signed_distance`, you need conditional logic without using `if`-statements, which is accomplished with `math.where`. You can also use `math.all` to deal with the case where one of the points is equal to a row of the `location` `Tensor`.

In [ ]:
class Triangle(Geometry):
    ...

## 3. Creating the scene
Construct a scene given the sketch in Figure 1, using your `Triangle` primitive and `Box`. For your domain, use bounds from `(0, 0)` to `(100, 100)` with a resolution of `100x100`. Your velocity field should be a `StaggeredGrid` initialized as `(0., 0.)` everywhere, but should have boundary conditions of `(0., 0.5)` on the bottom boundary, `GRAD_ZERO` on top, and zero on the sides.

Your obstacle geometries should be in each their `Obstacle` in a tuple, which you can pass to `fluid.make_incompressible`.
Simulate the incompressible fluid with `advect.semi_lagrangian` and `fluid.make_incompressible`.

Run 100 simulation steps with `dt=1.` and report on what you see when plotting the pressure and the velocity field at the final timestep.
At this point you use `@jit_compile` to speed up your simulation process.
Wrap your loop in a `tqdm` progress bar to make sure it will terminate. If this is still too slow on your computer you can reduce the resolution of the domain.

Hint: Search through the files in `demos/`.

In [ ]:
# Write your code here...

## 4. Optimizing the inflow velocity
Wrap the scene construction and simulation in a function that takes only the bottom boundary inflow velocity as an argument. After completing the 100 steps of simulation, sum up the velocity on the top boundary, and get the y-component of the mean velocity vector. Your function will return the squared error of this mean velocity vector and the target velocity of 1.0.

Use `math.gradient` to compute the gradient function of your custom defined function, and create a loop in which you perform gradient descent on the initial value for `y`.
Does it converge where you expected? Why or why not?

In [ ]:
# Write your code here...